In [1]:
!pip install datasets
!pip install git+https://github.com/google-research/bleurt.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 34.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-bui

https://github.com/google-research/bleurt/blob/master/checkpoints.md

https://docs.google.com/spreadsheets/d/1UUE_1JkLyexHOda3SqMBwsw9m2GG7zZMk7ChxglV5hA/edit?usp=sharing



In [2]:
from datasets import load_metric
import pandas as pd

gold_references = ['A 38 year old woman complains of right sided sharp and stabbing chest pain since 1 day.',
                   'A 38 year old woman complains of right sided sharp and stabbing chest pain since 1 day.']

our_predictions = ['A 38 year old lady complains of right sided sharp and stabbing chest pain since 1 day.',
                   '22 year old lady came on the platform to discuss results of her PAP smear done 1 month ago.']

metric = load_metric('bleurt')
metric.add_batch(predictions = our_predictions, references = gold_references)
score = metric.compute()
print(score)

<ipython-input-2-057538025f66>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('bleurt')


{'scores': [0.7759575247764587, -1.1086459159851074]}


Default (bleurt-base-128) = {'scores': [0.7759575247764587, -1.1086459159851074}

bleurt-base-512 = {'scores': [0.8908399939537048, -1.1607341766357422]} - 405M

bleurt-large-512 = {'scores': [1.0301593542099, -1.083858609199524]}



bleurt-20-d3 = {'scores': [0.780153751373291, 0.31089088320732117]}

bleurt-20-d12 = {'scores': [0.9019151926040649, 0.3302873373031616]} - 625M

bleurt-20 = {'scores': [0.924488365650177, 0.3025791645050049]}




In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def remove_name(all_mr, all_ref):
    unnamed_ref = []
    for mr, ref in zip(all_mr, all_ref):
        for relation in mr.split(' | '):
            attr, value = relation.split('=')
            attr = attr.strip()
            value = value.strip()
            if attr == 'name':
                ref = ref.replace(value, 'nameVariable')
        unnamed_ref.append(ref)
    return unnamed_ref

def viggo_eval(filepath, named=True, model='bleurt-base-128'):
    def best_score(s1, s2, s3):
        return max(s1, s2, s3)

    df = pd.read_csv(filepath)
    df_ref = pd.read_csv('/content/drive/MyDrive/KG-NLG Jurassic/personage_outputs/viggo_test100_refs.csv')
    generated = list(df["generated"])

    ref0 = list(df_ref["0"])
    ref1 = list(df_ref["1"])
    ref2= list(df_ref["2"])
    if named == False:
        df_mr = pd.read_csv('/content/drive/MyDrive/KG-NLG Jurassic/personage_outputs/viggo-test-new-all.csv')
        mr = list(df_mr["mr"])
        ref0 = remove_name(mr, ref0)
        ref1 = remove_name(mr, ref1)
        ref2 = remove_name(mr, ref2)

    metric = load_metric('bleurt', model)

    metric.add_batch(predictions=generated, references=ref0)
    score0 = metric.compute()
    df0 = pd.DataFrame.from_dict(score0)

    metric.add_batch(predictions=generated, references=ref1)
    score1 = metric.compute()
    df1 = pd.DataFrame.from_dict(score1)

    metric.add_batch(predictions=generated, references=ref2)
    score2 = metric.compute()
    df2 = pd.DataFrame.from_dict(score2)

    result = pd.concat([df0, df1, df2], axis=1)
    result.columns = ['score0', 'score1', 'score2']
    result['best_score'] = result.apply(
        lambda x: best_score(x.score0, x.score1, x.score2), axis=1)
    
    average_score = sum(result['best_score'])/len(result['best_score'])
    print(f'Average_score: {round(average_score, 3)}')
    return result

def rnnlg_eval(filepath, named=True, model='bleurt-base-128'):
    def best_score(s1, s2):
        return max(s1, s2)

    df = pd.read_csv(filepath)
    generated = list(df["generated"])

    ref0 = list(df["ref1"])
    ref1 = list(df["ref2"])
    if named == False:
        df1 = pd.read_csv('/content/drive/MyDrive/KG-NLG Jurassic/personage_outputs/rnnlg_resturant_test.csv')
        mr = list(df1["relations"])
        ref0 = remove_name(mr, ref0)
        ref1 = remove_name(mr, ref1)

    metric = load_metric('bleurt', model)

    metric.add_batch(predictions=generated, references=ref0)
    score0 = metric.compute()
    df0 = pd.DataFrame.from_dict(score0)

    metric.add_batch(predictions=generated, references=ref1)
    score1 = metric.compute()
    df1 = pd.DataFrame.from_dict(score1)

    result = pd.concat([df0, df1], axis=1)
    result.columns = ['score0', 'score1']
    result['best_score'] = result.apply(
        lambda x: best_score(x.score0, x.score1), axis=1)
    
    average_score = sum(result['best_score'])/len(result['best_score'])
    print(f'Average_score: {round(average_score, 3)}')
    return result

def personage_eval(filepath, named=True, model='bleurt-base-128'):

    df = pd.read_csv(filepath)
    ref = list(df["realization"])
    generated = list(df["generated"])

    metric = load_metric('bleurt', model)
    metric.add_batch(predictions=generated, references=ref)
    score = metric.compute()

    result = pd.DataFrame.from_dict(score)

    average_score = sum(result['score'])/len(result['score'])
    print(f'Average_score: {round(average_score, 3)}')

    return result

In [5]:
def tv_eval(filepath, named=True, model='bleurt-base-128'):
    def best_score(s1, s2, s3):
        return max(s1, s2, s3)

    df = pd.read_csv(filepath)
    #df_ref = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/NLP244_TV_chatgpt_90_responses.csv')
    generated = list(df["text"])

    """ref0 = list(df_ref["0"])
    ref1 = list(df_ref["1"])
    ref2= list(df_ref["2"])"""
    #if named == False:
    df_mr = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/Jurassic/Jurassic_Outputs.csv')
    mr = list(df_mr["mr"])
    """ref0 = remove_name(mr, ref0)
    ref1 = remove_name(mr, ref1)
    ref2 = remove_name(mr, ref2)"""

    metric = load_metric('bleurt', model)

    metric.add_batch(predictions=generated, references=mr)
    score0 = metric.compute()
    df0 = pd.DataFrame.from_dict(score0)

    """metric.add_batch(predictions=generated, references=ref1)
    score1 = metric.compute()
    df1 = pd.DataFrame.from_dict(score1)

    metric.add_batch(predictions=generated, references=ref2)
    score2 = metric.compute()
    df2 = pd.DataFrame.from_dict(score2)"""

    result = pd.concat([df0, df0, df0], axis=1)
    result.columns = ['score0', 'score1', 'score2']
    result['best_score'] = result.apply(
        lambda x: best_score(x.score0, x.score1, x.score2), axis=1)
    
    average_score = sum(result['best_score'])/len(result['best_score'])
    print(f'Average_score: {round(average_score, 3)}')
    return result
    
    average_score = sum(result['best_score'])/len(result['best_score'])
    print(f'Average_score: {round(average_score, 3)}')
    return result

In [6]:
ex_n = 'ex44'
#dir = f'/content/drive/MyDrive/KG-NLG Jurassic/personage_outputs/{ex_n}'
filepath = '/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/Jurassic/Jurassic_Outputs.csv'

result = tv_eval(filepath, named=True, model='bleurt-base-128')
result.to_csv( '/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/Jurassic/Jurassic_custom_output_scores.csv', index=None)
    

Average_score: -1.015
